<a href="https://colab.research.google.com/github/ajitnova/Automation/blob/master/fno.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
pip install --upgrade kiteconnect

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 2.5 MB/s 
     |████████████████████████████████| 771 kB 11.0 MB/s 
     |████████████████████████████████| 4.1 MB 37.7 MB/s 
     |████████████████████████████████| 251 kB 51.5 MB/s 
     |████████████████████████████████| 3.1 MB 36.7 MB/s 
     |████████████████████████████████| 74 kB 3.5 MB/s 


In [4]:
import logging
from kiteconnect import KiteConnect

logging.basicConfig(level=logging.DEBUG)
kite = KiteConnect(api_key="00j0ceqzr4aslm39")



In [5]:
#https://kite.zerodha.com/connect/login?v=3&api_key=00j0ceqzr4aslm39
request_token = 'aHUWstJG7Qn59ysQdXwP72b7vJwBVdtz'
api_secret = 'qadl2j9i31ukdom2by63eevxhfg6wcma'

data = kite.generate_session(request_token, api_secret)
kite.set_access_token(data["access_token"])

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "POST /session/token HTTP/1.1" 200 None


In [31]:
from datetime import date


global_option_instrument = {}

def nifty_option_instrument():
  options = kite.instruments('NFO')

  latest_expiry = [options[i] for i in range(len(options)) if options[i]['expiry'] == date(2022, 7, 21)]
  nifty_Spot = kite.ltp('NSE:NIFTY 50')['NSE:NIFTY 50']['last_price']

  itm_ce = [latest_expiry[i] for i in range(len(latest_expiry)) if latest_expiry[i]['strike'] - nifty_Spot < 0 and latest_expiry[i]['name'] == 'NIFTY' and latest_expiry[i]['instrument_type'] == 'CE']
  itm_pe = [latest_expiry[i] for i in range(len(latest_expiry)) if latest_expiry[i]['strike'] - nifty_Spot > 0 and latest_expiry[i]['name'] == 'NIFTY' and latest_expiry[i]['instrument_type'] == 'PE']

  itm_ce_vol = 0
  itm_pe_vol = 0

  trading_ce_quote = ''
  trading_pe_quote = ''

  for ce in itm_ce:
    symbol = 'NFO:' + ce['tradingsymbol']
    quote = kite.quote(symbol)
    if quote[symbol]['volume'] > itm_ce_vol:
      itm_ce_vol = quote[symbol]['volume']
      trading_ce_quote = ce['tradingsymbol']

  for pe in itm_pe:
    symbol = 'NFO:' + pe['tradingsymbol']
    quote = kite.quote(symbol)
    if quote[symbol]['volume'] > itm_pe_vol:
      itm_pe_vol = quote[symbol]['volume']
      trading_pe_quote = pe['tradingsymbol']

  return {'CE': trading_ce_quote, 'PE': trading_pe_quote}

global_option_instrument = nifty_option_instrument()




DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/NFO HTTP/1.1" 200 837257
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote/ltp?i=NSE%3ANIFTY+50 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote?i=NFO%3ANIFTY2272113800CE HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote?i=NFO%3ANIFTY2272113850CE HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote?i=NFO%3ANIFTY2272113900CE HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting n

In [6]:
import time as st

def nifty_option_decision(period, sleep_time):
  trading_pe_quote = global_option_instrument['PE']
  trading_ce_quote = global_option_instrument['CE']

  pe = {}
  ce = {}

  pe_price_change = []
  ce_price_change = []

  for i in range(period):
    pe = kite.quote('NFO:{}'.format(trading_pe_quote))
    ce = kite.quote('NFO:{}'.format(trading_ce_quote))

    pe_price_change.append(pe['NFO:{}'.format(trading_pe_quote)]['last_price'])
    ce_price_change.append(ce['NFO:{}'.format(trading_ce_quote)]['last_price'])

    st.sleep(sleep_time)

  diff_pe_price = []
  for idx, price in enumerate(pe_price_change):
    if idx == 0:
      continue
    else:
      diff = price - pe_price_change[idx - 1]
      diff_pe_price.append(diff)
  
  diff_ce_price = []
  for idx, price in enumerate(ce_price_change):
    if idx == 0:
      continue
    else:
      diff = price - ce_price_change[idx - 1]
      diff_ce_price.append(diff)

  if pe['NFO:{}'.format(trading_pe_quote)]['last_price'] > pe['NFO:{}'.format(trading_pe_quote)]['average_price'] and sum(diff_pe_price) > 0:
    return {"PE": trading_pe_quote}

  if ce['NFO:{}'.format(trading_ce_quote)]['last_price'] > ce['NFO:{}'.format(trading_ce_quote)]['average_price'] and sum(diff_ce_price) > 0:
    return {"CE": trading_ce_quote}
  

spot_prev_avg = []

def nifty_spot_decision(period, sleep_time):
  
  spot_price = []

  for i in range(period):
    spot = kite.ltp('NSE:NIFTY 50')
    spot_price.append(spot['NSE:NIFTY 50']['last_price'])
    st.sleep(sleep_time)

  spot_price_change = []
  for idx, price in enumerate(spot_price[1:]):
    if idx == 0:
      continue
    else:
      diff = price - spot_price[idx - 1]
      spot_price_change.append(diff)

  spot_avg = round(sum(spot_price)/len(spot_price), 2)
  
  global spot_prev_avg
  spot_prev_avg.append(spot_avg)


  if len(spot_prev_avg) >= 3:
    if sum(spot_price_change) > 0 and all([spot_price[-1] > i for i in spot_prev_avg[-3:]]):
      return "CE"
    if sum(spot_price_change) < 0 and all([spot_price[-1] < i for i in spot_prev_avg[-3:]]):
      return "PE"

In [10]:
nifty_spot_decision(10, 1)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote/ltp?i=NSE%3ANIFTY+50 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote/ltp?i=NSE%3ANIFTY+50 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote/ltp?i=NSE%3ANIFTY+50 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote/ltp?i=NSE%3ANIFTY+50 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /quote/ltp?i=NSE%3ANIFTY+50 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTT

In [43]:
from datetime import datetime, time
from multiprocessing.pool import ThreadPool

def trade(symbol, quantity):
  try:
    kite.place_order(variety='regular', exchange='NFO', tradingsymbol=symbol, transaction_type='BUY', quantity=quantity, product='NRML', order_type='MARKET')
  except Exception as e:
    pass

def check_position(spot_decision):
  positions = kite.positions()
  for position in positions['net']:
    if spot_decision and spot_decision != position['tradingsymbol'][-2:]:
        try:
          kite.place_order(variety='regular', exchange='NFO', tradingsymbol=position['tradingsymbol'], transaction_type='SELL', quantity=position['quantity'], product=position['product'], order_type='MARKET')
        except Exception as e:
          pass

  positions = kite.positions()
  for position in positions['net']:
    if (position['last_price'] - position['average_price']) /  position['average_price'] * 100 <= -5 :
      try:
        kite.place_order(variety='regular', exchange='NFO', tradingsymbol=position['tradingsymbol'], transaction_type='SELL', quantity=position['quantity'], product=position['product'], order_type='MARKET')
      except:
        pass
    if (position['last_price'] - position['average_price']) /  position['average_price'] * 100 >= 10 :
      try:
        kite.place_order(variety='regular', exchange='NFO', tradingsymbol=position['tradingsymbol'], transaction_type='SELL', quantity=position['quantity'], product=position['product'], order_type='MARKET')
      except:
        pass


def start_trading():
  now = datetime.now().time()

  open_time_str = '09::15::00'
  close_time_str = '15::30::00'

  open_time = datetime.strptime(open_time_str, '%H::%M::%S').time()
  close_time = datetime.strptime(close_time_str, '%H::%M::%S').time()

  quantity = 50
  period = 6
  sleep_time = 10

  while True: #now < close_time and now > open_time:
    
    pool = ThreadPool(processes=2)
    spot_decision_thread = pool.apply_async(nifty_spot_decision, (period, sleep_time))
    option_decision_thread = pool.apply_async(nifty_option_decision, (period, sleep_time))

    spot_decision = spot_decision_thread.get()
    option_decision = option_decision_thread.get()

    print(spot_decision)
    print(option_decision)
    
    if spot_decision and option_decision:
      if spot_decision == list(option_decision.keys())[0]:
        trade(symbol = list(option_decision.values())[0], quantity=quantity)
    check_position(spot_decision)

In [11]:
positions = kite.positions()

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /portfolio/positions HTTP/1.1" 200 None


In [39]:
positions = kite.positions()

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /portfolio/positions HTTP/1.1" 200 787


In [12]:
positions

{'day': [{'average_price': 0,
   'buy_m2m': 374602.5,
   'buy_price': 197.1592105263158,
   'buy_quantity': 1900,
   'buy_value': 374602.5,
   'close_price': 0,
   'day_buy_price': 197.1592105263158,
   'day_buy_quantity': 1900,
   'day_buy_value': 374602.5,
   'day_sell_price': 196.0013157894737,
   'day_sell_quantity': 1900,
   'day_sell_value': 372402.5,
   'exchange': 'NFO',
   'instrument_token': 13904642,
   'last_price': 224.25,
   'm2m': -2200,
   'multiplier': 1,
   'overnight_quantity': 0,
   'pnl': -2200,
   'product': 'NRML',
   'quantity': 0,
   'realised': 0,
   'sell_m2m': 372402.5,
   'sell_price': 196.0013157894737,
   'sell_quantity': 1900,
   'sell_value': 372402.5,
   'tradingsymbol': 'NIFTY2272115900CE',
   'unrealised': -2200,
   'value': -2200},
  {'average_price': 0,
   'buy_m2m': 5960,
   'buy_price': 119.2,
   'buy_quantity': 50,
   'buy_value': 5960,
   'close_price': 0,
   'day_buy_price': 119.2,
   'day_buy_quantity': 50,
   'day_buy_value': 5960,
   'day_s